In [1]:
import os
import sys
import argparse
import torch
import numpy as np
from random import shuffle
from collections import OrderedDict
import dataloaders.base
from torchvision import transforms
from dataloaders.datasetGen import SplitGen, PermutedGen, CORe50Gen, datasets4Gen, officehomeGen
import agents
import wandb

In [12]:
import torch
import torch.nn as nn
class FNN(nn.Module):
    def __init__(self) -> None:
        super(FNN,self).__init__()
        self.linear = nn.Linear(512,65)
    def forward(self,x):
        x= x.reshape([x.shape[0],x.shape[1]])
        return self.linear(x)

a = torch.rand(32,512,1,1)

fun=FNN()
fun(a)

tensor([[-0.4940,  0.3151, -0.1139,  ...,  0.0192,  0.5133, -0.0893],
        [-0.7008,  0.0009, -0.1303,  ...,  0.1174,  0.5438, -0.2427],
        [-0.3556,  0.0462,  0.1400,  ...,  0.2445,  0.5833, -0.0373],
        ...,
        [-0.3322,  0.1010,  0.4176,  ...,  0.3178,  0.6193,  0.0287],
        [-0.5742, -0.3458, -0.1731,  ..., -0.0173,  0.5325,  0.2360],
        [-0.3984,  0.0354, -0.1382,  ...,  0.2182,  0.4446, -0.0215]],
       grad_fn=<AddmmBackward>)

In [2]:
import torchvision
import torch.nn as nn
model18 = torchvision.models.resnet18(pretrained=True)
print(model18)
numFit = model18.fc.in_features
model18.fc = nn.Linear(numFit, 65)
print(model18)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [2]:
train_dataset_splits, val_dataset_splits, task_output_space = officehomeGen()

In [4]:
for data in val_dataset_splits['2']:
    print(data[0].shape,data[1],data[2]) # need input, target(label), task
    break

torch.Size([3, 224, 224]) 0 2


In [2]:
dataloaders.base.__dict__['CIFAR100']

<function dataloaders.base.CIFAR100(dataroot, train_aug=False)>

In [46]:
train_dataset, val_dataset = dataloaders.base.__dict__['CIFAR100']('data', True)

Files already downloaded and verified
Files already downloaded and verified


In [47]:
print(train_dataset)
print(val_dataset.dataset)

Dataset CIFAR100
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.507, 0.487, 0.441], std=[0.267, 0.256, 0.276])
           )


In [30]:
train_dataset_splits, val_dataset_splits, task_output_space = SplitGen(train_dataset, val_dataset,
                                                                          first_split_sz=20,
                                                                          other_split_sz=20,
                                                                          rand_split=False,
                                                                          remap_class=not False)

split_boundaries: [0, 20, 40, 60, 80, 100]
{'1': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], '2': [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], '3': [40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], '4': [60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79], '5': [80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]}


In [32]:
print(train_dataset_splits['1'].dataset)
print(train_dataset_splits['1'].dataset.dataset)
print(train_dataset_splits['1'])
print(train_dataset_splits['2'].dataset.__len__()) # 每个都是10000个
print(train_dataset_splits['1'].__len__())
print(train_dataset_splits)

10000
10000
{'1': <dataloaders.wrapper.AppendName object at 0x7f10330926d0>, '2': <dataloaders.wrapper.AppendName object at 0x7f1029ad5310>, '3': <dataloaders.wrapper.AppendName object at 0x7f1032f28590>, '4': <dataloaders.wrapper.AppendName object at 0x7f1032f28a90>, '5': <dataloaders.wrapper.AppendName object at 0x7f1032f2b1d0>}


In [8]:
for data in train_dataset_splits['2']:
    print(data[0].shape,data[1],data[2]) # need input, target(label), task
    break

torch.Size([3, 32, 32]) 0 2


In [9]:
val_dataset_splits

{'1': <dataloaders.wrapper.AppendName at 0x7f109749a090>,
 '2': <dataloaders.wrapper.AppendName at 0x7f103307e610>,
 '3': <dataloaders.wrapper.AppendName at 0x7f1033092510>,
 '4': <dataloaders.wrapper.AppendName at 0x7f1033092050>,
 '5': <dataloaders.wrapper.AppendName at 0x7f1097497710>}

In [10]:
task_output_space

{'1': 20, '2': 20, '3': 20, '4': 20, '5': 20}

In [48]:
train_dataset_splits, val_dataset_splits, task_output_space = PermutedGen(train_dataset, val_dataset,
                                                                             10,
                                                                             remap_class=False)

In [49]:
train_dataset_splits

{'1': <dataloaders.wrapper.AppendName at 0x7f1032c84110>,
 '2': <dataloaders.wrapper.AppendName at 0x7f1032f28410>,
 '3': <dataloaders.wrapper.AppendName at 0x7f10330a4d50>,
 '4': <dataloaders.wrapper.AppendName at 0x7f10330a4190>,
 '5': <dataloaders.wrapper.AppendName at 0x7f1032f40bd0>,
 '6': <dataloaders.wrapper.AppendName at 0x7f1029520750>,
 '7': <dataloaders.wrapper.AppendName at 0x7f1029520910>,
 '8': <dataloaders.wrapper.AppendName at 0x7f1029520a50>,
 '9': <dataloaders.wrapper.AppendName at 0x7f1029520b90>,
 '10': <dataloaders.wrapper.AppendName at 0x7f1029520cd0>}

In [50]:
val_dataset_splits

{'1': <dataloaders.wrapper.AppendName at 0x7f1032c84090>,
 '2': <dataloaders.wrapper.AppendName at 0x7f1032f285d0>,
 '3': <dataloaders.wrapper.AppendName at 0x7f10330a45d0>,
 '4': <dataloaders.wrapper.AppendName at 0x7f10330a4d10>,
 '5': <dataloaders.wrapper.AppendName at 0x7f1029520790>,
 '6': <dataloaders.wrapper.AppendName at 0x7f1029520850>,
 '7': <dataloaders.wrapper.AppendName at 0x7f1029520990>,
 '8': <dataloaders.wrapper.AppendName at 0x7f1029520ad0>,
 '9': <dataloaders.wrapper.AppendName at 0x7f1029520c10>,
 '10': <dataloaders.wrapper.AppendName at 0x7f1029520d50>}

In [51]:
task_output_space

{'1': 100,
 '2': 100,
 '3': 100,
 '4': 100,
 '5': 100,
 '6': 100,
 '7': 100,
 '8': 100,
 '9': 100,
 '10': 100}

In [36]:
train_dataset_splits['1']

In [38]:
for data in train_dataset_splits['1']:
    print(data[0].shape,data[1],data[2]) # need input, target(label), task
    

torch.Size([3, 32, 32]) 19 1
torch.Size([3, 32, 32]) 29 1
torch.Size([3, 32, 32]) 0 1
torch.Size([3, 32, 32]) 11 1
torch.Size([3, 32, 32]) 1 1
torch.Size([3, 32, 32]) 86 1
torch.Size([3, 32, 32]) 90 1
torch.Size([3, 32, 32]) 28 1
torch.Size([3, 32, 32]) 23 1
torch.Size([3, 32, 32]) 31 1
torch.Size([3, 32, 32]) 39 1
torch.Size([3, 32, 32]) 96 1
torch.Size([3, 32, 32]) 82 1
torch.Size([3, 32, 32]) 17 1
torch.Size([3, 32, 32]) 71 1
torch.Size([3, 32, 32]) 39 1
torch.Size([3, 32, 32]) 8 1
torch.Size([3, 32, 32]) 97 1
torch.Size([3, 32, 32]) 80 1
torch.Size([3, 32, 32]) 71 1
torch.Size([3, 32, 32]) 74 1
torch.Size([3, 32, 32]) 59 1
torch.Size([3, 32, 32]) 70 1
torch.Size([3, 32, 32]) 87 1
torch.Size([3, 32, 32]) 59 1
torch.Size([3, 32, 32]) 84 1
torch.Size([3, 32, 32]) 64 1
torch.Size([3, 32, 32]) 52 1
torch.Size([3, 32, 32]) 42 1
torch.Size([3, 32, 32]) 64 1
torch.Size([3, 32, 32]) 8 1
torch.Size([3, 32, 32]) 17 1
torch.Size([3, 32, 32]) 47 1
torch.Size([3, 32, 32]) 65 1
torch.Size([3, 32,

In [2]:
train_dataset_splits, val_dataset_splits, task_output_space = CORe50Gen()

In [3]:
train_dataset_splits

{'1': <dataloaders.wrapper.AppendName at 0x7f5ea4ed7b10>,
 '2': <dataloaders.wrapper.AppendName at 0x7f5ea4ed7b50>,
 '3': <dataloaders.wrapper.AppendName at 0x7f5ea4ed7e10>,
 '4': <dataloaders.wrapper.AppendName at 0x7f5ea4ed7490>,
 '5': <dataloaders.wrapper.AppendName at 0x7f5ea4ed7d50>,
 '6': <dataloaders.wrapper.AppendName at 0x7f5ea4ed7c50>,
 '7': <dataloaders.wrapper.AppendName at 0x7f5ea4ed7c10>,
 '8': <dataloaders.wrapper.AppendName at 0x7f5ea4ed7dd0>}

In [2]:
train_dataset_splits, val_dataset_splits, task_output_space = datasets4Gen()

In [ ]:
for data in train_dataset_splits['1']:
    print(data[0].shape,data[1],data[2]) # need input, target(label), task
    

In [5]:
train_dataset_splits.values()

dict_values([<dataloaders.wrapper.AppendName object at 0x7f5ea4ed7b10>, <dataloaders.wrapper.AppendName object at 0x7f5ea4ed7b50>, <dataloaders.wrapper.AppendName object at 0x7f5ea4ed7e10>, <dataloaders.wrapper.AppendName object at 0x7f5ea4ed7490>, <dataloaders.wrapper.AppendName object at 0x7f5ea4ed7d50>, <dataloaders.wrapper.AppendName object at 0x7f5ea4ed7c50>, <dataloaders.wrapper.AppendName object at 0x7f5ea4ed7c10>, <dataloaders.wrapper.AppendName object at 0x7f5ea4ed7dd0>])

In [4]:
for data in train_dataset_splits['1']:
    print(data[0].shape,data[1],data[2]) # need input, target(label), task
    

torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size([3, 224, 224]) 0 1
torch.Size

In [39]:
from torchvision.datasets import ImageFolder
from torchvision import datasets
import torchvision

In [40]:
data = ImageFolder(root='/data1/zhaohongbo/core50_128x128/s1/',transform=torchvision.transforms.ToTensor())

In [13]:
data

Dataset ImageFolder
    Number of datapoints: 14989
    Root location: /data1/zhaohongbo/core50_128x128/s1/
    StandardTransform
Transform: ToTensor()

In [41]:
from dataloaders.wrapper import AppendName
train_dataset_splits = {}
train_dataset_splits['1'] = AppendName(data,str(1))

In [42]:
train_dataset_splits['1'].dataset

Dataset ImageFolder
    Number of datapoints: 14989
    Root location: /data1/zhaohongbo/core50_128x128/s1/
    StandardTransform
Transform: ToTensor()

In [45]:
for data in train_dataset_splits['1']:
    print(data) # need input, target(label), task
    break

(tensor([[[0.7608, 0.7647, 0.7725,  ..., 0.7412, 0.7647, 0.7765],
         [0.7686, 0.7686, 0.7765,  ..., 0.7412, 0.7647, 0.7725],
         [0.7725, 0.7686, 0.7725,  ..., 0.7490, 0.7647, 0.7686],
         ...,
         [0.6902, 0.7020, 0.7059,  ..., 0.8196, 0.8196, 0.8196],
         [0.6941, 0.7059, 0.7098,  ..., 0.8235, 0.8196, 0.8118],
         [0.6980, 0.7059, 0.6980,  ..., 0.8235, 0.8078, 0.7961]],

        [[0.7490, 0.7529, 0.7608,  ..., 0.7294, 0.7529, 0.7647],
         [0.7490, 0.7490, 0.7569,  ..., 0.7294, 0.7529, 0.7608],
         [0.7529, 0.7490, 0.7529,  ..., 0.7373, 0.7529, 0.7569],
         ...,
         [0.6863, 0.6980, 0.7020,  ..., 0.8039, 0.8157, 0.8118],
         [0.6902, 0.7020, 0.7059,  ..., 0.8078, 0.8157, 0.8039],
         [0.6941, 0.7020, 0.7020,  ..., 0.8118, 0.8039, 0.7882]],

        [[0.7647, 0.7686, 0.7765,  ..., 0.7412, 0.7647, 0.7804],
         [0.7647, 0.7647, 0.7725,  ..., 0.7451, 0.7686, 0.7765],
         [0.7686, 0.7647, 0.7686,  ..., 0.7529, 0.7686, 0

In [14]:
data[0][0],data[0][1] # 第一维是第几张图，第二维为1返回label，第二维0返回数组

(tensor([[[0.7608, 0.7647, 0.7725,  ..., 0.7412, 0.7647, 0.7765],
          [0.7686, 0.7686, 0.7765,  ..., 0.7412, 0.7647, 0.7725],
          [0.7725, 0.7686, 0.7725,  ..., 0.7490, 0.7647, 0.7686],
          ...,
          [0.6902, 0.7020, 0.7059,  ..., 0.8196, 0.8196, 0.8196],
          [0.6941, 0.7059, 0.7098,  ..., 0.8235, 0.8196, 0.8118],
          [0.6980, 0.7059, 0.6980,  ..., 0.8235, 0.8078, 0.7961]],
 
         [[0.7490, 0.7529, 0.7608,  ..., 0.7294, 0.7529, 0.7647],
          [0.7490, 0.7490, 0.7569,  ..., 0.7294, 0.7529, 0.7608],
          [0.7529, 0.7490, 0.7529,  ..., 0.7373, 0.7529, 0.7569],
          ...,
          [0.6863, 0.6980, 0.7020,  ..., 0.8039, 0.8157, 0.8118],
          [0.6902, 0.7020, 0.7059,  ..., 0.8078, 0.8157, 0.8039],
          [0.6941, 0.7020, 0.7020,  ..., 0.8118, 0.8039, 0.7882]],
 
         [[0.7647, 0.7686, 0.7765,  ..., 0.7412, 0.7647, 0.7804],
          [0.7647, 0.7647, 0.7725,  ..., 0.7451, 0.7686, 0.7765],
          [0.7686, 0.7647, 0.7686,  ...,

In [ ]:
from torch.utils.data import dataset

class COReDataloader(dataset.Dataset):
    def __init__(self) -> None:
        super().__init__()
    def __getitem__(self, index):
        pass
    def __len__(self):
        pass

In [16]:
import torch
from torch.utils.data import _utils
dataloader = torch.utils.data.DataLoader(dataset = data,batch_size=256,shuffle=True)

In [19]:
dataloader
for data in dataloader:
    print(data[0].shape,data[1].shape)

torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 128, 128]) torch.Size([256])
torch.Size([256, 3, 

In [52]:
task_output_space = {}
for i in range(1,9):
        task_output_space[str(i)]=50
task_output_space

{'1': 50, '2': 50, '3': 50, '4': 50, '5': 50, '6': 50, '7': 50, '8': 50}

In [ ]:
dataloader

In [ ]:
len(data)

In [ ]:
type(data)

In [ ]:
data.classes

In [ ]:
data[10000][0]

In [ ]:
next(data)

In [ ]:
torch.LongTensor(10).fill_(-1)

In [ ]:
import torchvision
tdata=torchvision.datasets.CIFAR100(
        root='data',
        train=True,
        download=True)

In [ ]:
tdata

In [ ]:
type(tdata)

In [ ]:
tdata

In [3]:
from collections import OrderedDict
a=OrderedDict([('1', OrderedDict([('1', 63.9972385226096), ('2', 24.231964100793924), ('3', 0.7248878149810148), ('4', 11.149464963755609)])), ('2', OrderedDict([('2', 82.14343795739144), ('3', 0.47162140185396), ('4', 18.246869409660107)])), ('3', OrderedDict([('3', 74.2), ('4', 4.45)])), ('4', OrderedDict([('4', 75.04041785847531)]))])

In [4]:
a

OrderedDict([('1',
              OrderedDict([('1', 63.9972385226096),
                           ('2', 24.231964100793924),
                           ('3', 0.7248878149810148),
                           ('4', 11.149464963755609)])),
             ('2',
              OrderedDict([('2', 82.14343795739144),
                           ('3', 0.47162140185396),
                           ('4', 18.246869409660107)])),
             ('3', OrderedDict([('3', 74.2), ('4', 4.45)])),
             ('4', OrderedDict([('4', 75.04041785847531)]))])

In [5]:
len(a)

4

In [11]:
(max(a['1'].values())-a['1']['4']+max(a['2'].values())-a['2']['4']+max(a['3'].values())-a['3']['4'])/4

46.62358552664633

In [12]:
a.keys()

odict_keys(['1', '2', '3', '4'])

In [18]:
lasttask=list(a.keys())[len(a)-1]
lasttask

'4'

In [20]:
for valtask in list(a.keys()):
    temp=max(a[valtask].values())-a[valtask][lasttask]
    print(temp)

52.84777355885399
63.896568547731334
69.75
0.0
